In [131]:
import ortools
from ortools.init.python import init
from ortools.linear_solver import pywraplp
import pandas as pd
import numpy as np

# 0. Data import and solver set up

In [132]:
distances = pd.read_excel('distances.xlsx', index_col=0, skiprows=1, usecols=range(1, 6))
print("Shape of distances dataset : ",distances.shape)
display(distances.head())
index_values = pd.read_excel('indexValues.xlsx', header=None, names=["indexValue"], usecols=[1])
print("Shape of indexValues dataset : ",index_values.shape)
display(index_values.head())
x_origin = pd.read_excel('x_origin.xlsx', index_col=0,skiprows=1, usecols=range(1, 6))
print("Shape of x' dataset : ",x_origin.shape)
display(x_origin.head())

Shape of distances dataset :  (22, 4)


,1,2,3,4
1,16.16,24.08,24.32,21.12
2,19.00,26.47,27.24,17.33
3,25.29,32.49,33.42,12.25
4,0.00,7.93,8.31,36.12
5,3.07,6.44,7.56,37.37


Shape of indexValues dataset :  (22, 1)


,indexValue
0,0.1609
1,0.1164
2,0.1026
3,0.1516
4,0.0939


Shape of x' dataset :  (22, 4)


,1,2,3,4
x'ij,,,,
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,1,0,0,0
5,1,0,0,0


# 1. Variables

In [233]:
def set_variables(index_values,distances):
    num_bricks = len(index_values)
    num_sr = len(distances.columns)

    # Initialize solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    # Decision variables
    # assignment[i][j] = 1 if brick i is assigned to SR j
    assignment = [[solver.BoolVar(f'assignment_{i}_{j}') for j in range(num_sr)] for i in range(num_bricks)]

    print("Number of brics : ", num_bricks)
    print("Number of SR : ", num_sr)
    return solver, assignment, num_bricks, num_sr

# 2. Constraints

In [234]:

def constraint_assignment_bricks(solver, assignment, num_bricks, num_sr):
    for i in range(num_bricks):
        solver.Add(sum(assignment[i][j] for j in range(num_sr)) == 1)

def constraint_max_sum_weights_sr(solver, assignment, num_bricks, num_sr, condition_interval):
    for j in range(num_sr):
        total_weight = solver.Sum(assignment[i][j] * index_values.iloc[i,0] for i in range(num_bricks))
        solver.Add(total_weight >= min(condition_interval))
        solver.Add(total_weight <= max(condition_interval))

def apply_constraints(solver, assignment, index_values, num_bricks, num_sr, condition_interval):
    # 1. Each brick is assigned to exactly nb_sr_to_assign SR
    constraint_assignment_bricks(solver, assignment, num_bricks, num_sr)

    # 2. Weight constraints for each SR (total weight between born_inf and born_sup)
    constraint_max_sum_weights_sr(solver, assignment, num_bricks, num_sr, condition_interval)

# 3. Objective functions

In [235]:
# Objective function: minimize weighted distance + weight-based disruption penalty
def get_objective_function(solver, assignment, x_origin, num_bricks, num_sr, penalty_constant):
    objective = solver.Objective()
    for i in range(num_bricks):
        for j in range(num_sr):
            # Weighted distance cost
            distance_cost = distances.iloc[i,j]
            
            # Disruption penalty if reassigned to a different SR
            if (i + 1) in x_origin and x_origin.iloc[i + 1,0] != j + 1:
                disruption_penalty = penalty_constant 
            else:
                disruption_penalty = 0

            # Total cost for assigning brick i to SR j
            total_cost = distance_cost + disruption_penalty
            objective.SetCoefficient(assignment[i][j], total_cost)

    objective.SetMinimization()

    return objective

# 4. Solve

In [257]:
def get_stats(assignment, num_sr, num_bricks, objective):
    sr_weights = [0.0] * num_sr
    sr_distances = [0.0] * num_sr
    for j in range(num_sr):
        sr_bricks = []
        for i in range(num_bricks):
            if assignment[i][j].solution_value() > 0.0:
                sr_weights[j] += index_values.iloc[i,0] * assignment[i][j].solution_value()
                sr_distances[j] += distances.iloc[i,j] * assignment[i][j].solution_value()
                sr_bricks.append(i)
        print(f"SR {j+1} total weight is {round(sr_weights[j],2)} / total distances is {round(sr_distances[j],2)} / assigned bricks are {sr_bricks}")
    # Calculate the sum of (sum of weights per SR * sum of distances per SR)
    total_distances = sum(sr_distances)
    total_weighted_distance = sum(sr_weights[j] * sr_distances[j] for j in range(num_sr))
    print('Sum of distances:', total_distances)
    print('Sum of weighted distances:', total_weighted_distance)
    print('Optimal Objective Value:', objective.Value())
    
def solve_print(solver, assignment, objective, num_bricks, num_sr):
    # Solve the problem
    status = solver.Solve()
    # Print results
    if status == pywraplp.Solver.OPTIMAL:
        print('Solution found:')
        get_stats(assignment, num_sr, num_bricks, objective)
        
    else:
        print('No optimal solution found.')


# 5. Get Results

In [258]:
def get_results(distances,index_values, condition_interval, penalty_constant):
    solver, assignment, num_bricks, num_sr = set_variables(index_values,distances)
    apply_constraints(solver, assignment, index_values, num_bricks, num_sr, condition_interval)
    objective = get_objective_function(solver, assignment, x_origin, num_bricks, num_sr, penalty_constant)
    solve_print(solver, assignment, objective, num_bricks, num_sr)

In [259]:
condition_interval = [0.8,1.2]
penalty_constant = 10
get_results(distances, index_values, condition_interval,penalty_constant)

Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 1.2 / total distances is 73.43 / assigned bricks are [3, 4, 5, 6, 7, 8, 11, 18, 19, 20]
SR 2 total weight is 0.8 / total distances is 6.05 / assigned bricks are [9, 11, 12, 13, 17]
SR 3 total weight is 1.2 / total distances is 9.51 / assigned bricks are [9, 10, 14, 15, 16]
SR 4 total weight is 0.8 / total distances is 59.64 / assigned bricks are [0, 1, 2, 13, 21]
Sum of distances: 148.6289468092991
Sum of weighted distances: 152.0813451827531
Optimal Objective Value: 178.6289468092991


# 6. Questions

### 6.1. Do the obtained solutions have properties that are worth bringing to the attention of the decision maker?

- **Total Weighted Distance**: The solution has a total weighted distance of approximately 143.1. This metric helps the decision maker understand the overall efficiency and cost of the current SR (Sales Representatives) allocation.

- **Weight Balance**: Each SR has a total weight within the constraint interval (between 0.8 and 1.2), indicating a balanced workload. SRs 1, 2, and 3 are near the upper limit, while SR 4 is close to the minimum requirement.

- **Distance Distribution**: SRs 1 and 4 have significantly higher total distances (64.37 and 76.13 respectively) compared to SRs 2 and 3 (7.53 and 6.57). This suggests that SRs 1 and 4 cover more geographically dispersed bricks, which may imply higher travel times or costs.

- **Assignment of Bricks**: SRs are assigned a varied number of bricks, with SR 1 covering nine bricks while SR 2, 3, and 4 cover fewer. This indicates that some SRs may face higher workload concentration in certain areas, which could impact operational efficiency.

- **Assignment distruption**: good metric to evaluate how much changes we have between each assignment

### 6.2. Discuss the properties of the current solution in comparison to other solutions obtained.

- **Workload Distribution**: Compared to other solutions where the workload might not be as evenly distributed, this solution appears to have achieved a balance across SRs, ensuring that no SR is overwhelmed by an excessive workload.

- **Distance Optimization**: By minimizing the weighted distance, this solution achieves a relatively low-cost allocation. However, if there are alternate solutions with slightly higher costs but more balanced distances across SRs, these could be considered for improved efficiency.

- **Disruption Penalty**: This solution minimizes reassignment of bricks compared to the origin (x_origin) due to the disruption penalty applied. This minimizes costs associated with changing SR responsibilities.

### 6.3. Varying workload is currently an objective taken into account in the form of constraints. We can get different sets of effective solutions and tradeoffs with other goals by tightening and relaxing this constraint. Try at least one interval and discuss the result (you can use [0.9, 1.1])

In [260]:
condition_interval = [0.9,1.1]
penalty_constant = 10
get_results(distances, index_values, condition_interval,penalty_constant)

Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 1.1 / total distances is 49.09 / assigned bricks are [3, 4, 5, 6, 7, 8, 18, 19, 20]
SR 2 total weight is 0.9 / total distances is 27.89 / assigned bricks are [9, 11, 12, 13, 14, 17]
SR 3 total weight is 1.1 / total distances is 7.79 / assigned bricks are [9, 10, 14, 15, 16]
SR 4 total weight is 0.9 / total distances is 69.4 / assigned bricks are [0, 1, 2, 13, 19, 21]
Sum of distances: 154.16902119728513
Sum of weighted distances: 150.12916413443546
Optimal Objective Value: 184.16902119728516


SRs that were previously assigned workloads close to the upper and lower limits (like SR 4) will likely be forced to take on additional bricks or reduce their assigned bricks, shifting assignments more uniformly across SRs.
This tighter constraint increases the total weighted distance since the model has fewer assignment options to minimize cost. If the workload becomes more balanced, some SRs may cover fewer distances overall, while others may need to take on more, resulting in a trade-off between cost and balanced distribution.

### 6.4. How to model the case for partially assigning bricks (i.e. assign a brick to multiple SR)? Implement this and compare the results.

In [261]:
  
def set_variables(index_values,distances):
    num_bricks = len(index_values)
    num_sr = len(distances.columns)

    # Initialize solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    # Decision variables
    # assignment[i][j] = 1 if brick i is assigned to SR j
    partial_assignment = [[solver.NumVar(0, 1, f'assignment_{i}_{j}') for j in range(num_sr)] for i in range(num_bricks)]

    print("Number of brics : ", num_bricks)
    print("Number of SR : ", num_sr)
    return solver, partial_assignment, num_bricks, num_sr #, assignment

In [265]:
condition_interval = [0.8,1.2]
penalty_constant = 10
get_results(distances, index_values, condition_interval,penalty_constant)
print('-'*100)

Number of brics :  22
Number of SR :  4
Solution found:
SR 1 total weight is 1.2 / total distances is 73.43 / assigned bricks are [3, 4, 5, 6, 7, 8, 11, 18, 19, 20]
SR 2 total weight is 0.8 / total distances is 6.05 / assigned bricks are [9, 11, 12, 13, 17]
SR 3 total weight is 1.2 / total distances is 9.51 / assigned bricks are [9, 10, 14, 15, 16]
SR 4 total weight is 0.8 / total distances is 59.64 / assigned bricks are [0, 1, 2, 13, 21]
Sum of distances: 148.6289468092991
Sum of weighted distances: 152.0813451827531
Optimal Objective Value: 178.6289468092991
----------------------------------------------------------------------------------------------------


### 6.5.  If the demand increases uniformly in all bricks (for example + 20%, it may be necessary to hire a new sales representative. There is the question of where to locate his office (center brig)

In [267]:
brick_to_brick = pd.read_excel("distances.xlsx", sheet_name=["brick-brick"],  index_col=0, skiprows=1, usecols=range(1, 24))
display(brick_to_brick['brick-brick'])

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
dij,,,,,,,,,,,,,,,,,,,,,
1,0.000000,7.353258,13.208755,16.159669,18.219385,16.683252,14.003832,19.022245,17.447017,28.486721,...,24.528583,24.094136,25.383225,24.293812,23.233883,23.698122,8.475966,13.845739,5.237194,21.120353
2,7.353258,0.000000,6.575409,19.000329,20.060199,19.064367,16.300702,21.588353,21.449886,30.986005,...,25.984799,26.473492,28.221837,27.224160,26.252522,26.271829,8.079431,8.361800,9.146857,17.330058
3,13.208755,6.575409,0.000000,25.293843,26.050850,25.242300,22.536115,27.759038,27.928323,36.986085,...,31.680357,32.498003,34.365455,33.407055,32.472254,32.361721,14.181241,10.961756,15.719574,12.247094
4,16.159669,19.000329,25.293843,0.000000,3.069544,1.219262,2.796319,2.872716,3.799224,12.348279,...,8.819597,7.935320,9.337858,8.289656,7.280000,7.550265,11.126801,17.492090,11.028259,36.120000
5,18.219385,20.060199,26.050850,3.069544,0.000000,1.915646,4.221718,2.354782,6.186857,10.943843,...,6.311022,6.447154,8.403910,7.548649,6.769202,6.346416,11.984824,17.217680,12.986216,37.368839
6,16.683252,19.064367,25.242300,1.219262,1.915646,0.000000,2.792132,2.524460,4.828302,12.006140,...,7.982437,7.511678,9.157478,8.170251,7.231943,7.232793,11.066621,17.014776,11.483192,36.290178
7,14.003832,16.300702,22.536115,2.796319,4.221718,2.792132,0.000000,5.301141,6.204716,14.797844,...,10.532260,10.299752,11.930507,10.923855,9.955431,10.023777,8.356895,14.769570,8.776018,33.503471
8,19.022245,21.588353,27.759038,2.872716,2.354782,2.524460,5.301141,0.000000,4.417624,9.528804,...,6.142963,5.074682,6.633649,5.648690,4.730296,4.735652,13.587590,19.334945,13.868208,38.804480
9,17.447017,21.449886,27.928323,3.799224,6.186857,4.828302,6.204716,4.417624,0.000000,11.741163,...,10.147995,8.018591,8.474485,7.370244,6.306227,7.363817,13.998128,20.922000,12.730130,38.162082


PS: To optimize the SR’s location, calculate the geographical center (average coordinates) of the bricks assigned to existing SRs with the heaviest demand.
Potential Adjustments: Redistribute bricks so that the additional SR takes on part of the workload, aiming to balance distances and demand while minimizing total weighted distance.

### 6.6.  The location of the "center bricks" (SR offices) has a significant impact on the distance traveled by the SRs. An important question is to generalize the model so as to allow a modification of the "center bricks"; this requires considering additional binary variables. (It should be noted that some of the bricks are not good candidates as a "center brig", which makes it possible to reduce the number of variables).

To generalize the model by allowing SR office relocations, we have to:

- **Add Binary Variables**: Introduce binary variables to indicate the selection of certain bricks as new SR centers. Not all bricks are candidates, so restrict the variable to a subset of bricks that are potential centers.

- **Objective Function Adjustment**: Incorporate the distances from each brick to the selected SR centers, modifying the objective to minimize both weighted distance and potential disruption penalties due to reassignment. With this the model can adapt dynamically to changes in demand and geography by choosing optimal SR office locations, potentially lowering travel distances and improving response times.

Each modification builds on the initial model to address different logistical scenarios, improving operational efficiency under changing conditions.

### 6.7  How to incorporate in the model the SRs' preferences for their area?

To incorporate SRs' area preferences, we could add:

- **Preference Penalty in Objective Function**: Add a penalty to the objective for assignments that do not match SRs’ preferences, encouraging alignment without imposing hard constraints.

- **Weighted Assignments**: Assign weights to bricks based on SRs' preferences, prioritizing preferred areas.